In [11]:
import tensorflow as tf
import pandas as pd
import pickle

In [2]:
rootpath = "data/bms-molecular-translation/"

In [3]:
train = pd.read_csv(rootpath + "train_labels.csv")

In [4]:
def get_train_file_path(image_id):
    return rootpath + "train/{}/{}/{}/{}.png".format(
        image_id[0], image_id[1], image_id[2], image_id 
    )

train['file_path'] = train['image_id'].apply(get_train_file_path)

print(f'train.shape: {train.shape}')
display(train.head())

train.shape: (2424186, 3)


,image_id,InChI,file_path
0,000011a64c74,InChI=1S/C13H20OS/c1-9(2)8-15-13-6-5-10(3)7-12...,data/bms-molecular-translation/train/0/0/0/000...
1,000019cc0cd2,InChI=1S/C21H30O4/c1-12(22)25-14-6-8-20(2)13(1...,data/bms-molecular-translation/train/0/0/0/000...
2,0000252b6d2b,InChI=1S/C24H23N5O4/c1-14-13-15(7-8-17(14)28-1...,data/bms-molecular-translation/train/0/0/0/000...
3,000026b49b7e,InChI=1S/C17H24N2O4S/c1-12(20)18-13(14-7-6-10-...,data/bms-molecular-translation/train/0/0/0/000...
4,000026fc6c36,InChI=1S/C10H19N3O2S/c1-15-10(14)12-8-4-6-13(7...,data/bms-molecular-translation/train/0/0/0/000...


In [5]:
def preprocess_label(InChI):
    return "<{}>".format(InChI.replace("InChI=1S/", ""))

In [6]:
train["InChI_clean"] = train["InChI"].apply(preprocess_label)

In [7]:
train.head()

,image_id,InChI,file_path,InChI_clean
0,000011a64c74,InChI=1S/C13H20OS/c1-9(2)8-15-13-6-5-10(3)7-12...,data/bms-molecular-translation/train/0/0/0/000...,<C13H20OS/c1-9(2)8-15-13-6-5-10(3)7-12(13)11(4...
1,000019cc0cd2,InChI=1S/C21H30O4/c1-12(22)25-14-6-8-20(2)13(1...,data/bms-molecular-translation/train/0/0/0/000...,<C21H30O4/c1-12(22)25-14-6-8-20(2)13(10-14)11-...
2,0000252b6d2b,InChI=1S/C24H23N5O4/c1-14-13-15(7-8-17(14)28-1...,data/bms-molecular-translation/train/0/0/0/000...,<C24H23N5O4/c1-14-13-15(7-8-17(14)28-12-10-20(...
3,000026b49b7e,InChI=1S/C17H24N2O4S/c1-12(20)18-13(14-7-6-10-...,data/bms-molecular-translation/train/0/0/0/000...,<C17H24N2O4S/c1-12(20)18-13(14-7-6-10-24-14)11...
4,000026fc6c36,InChI=1S/C10H19N3O2S/c1-15-10(14)12-8-4-6-13(7...,data/bms-molecular-translation/train/0/0/0/000...,<C10H19N3O2S/c1-15-10(14)12-8-4-6-13(7-8)5-2-3...


In [8]:
# tokenizing
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token="q", filters=' ', lower=False, char_level=True)
tokenizer.fit_on_texts(train["InChI_clean"].values)
tokenizer.word_index['q'] = 0
tokenizer.index_word[0] = 'q'

In [9]:
vocab_size = max(tokenizer.index_word.keys())+1
vocab_size

40

In [10]:
train_token = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(train["InChI_clean"].values), padding='post')
train_token.shape

(2424186, 396)

## Saving 

In [12]:
# saving
with open('data/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
# saving
with open('data/train_preprocessed.pickle', 'wb') as handle:
    pickle.dump({"paths": train["file_path"], "train_token": train_token}, handle, protocol=pickle.HIGHEST_PROTOCOL)